In [ ]:
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [27]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_CONTOURS

PARAMETROS MP Holistic

In [73]:
mp_holistic = mp.solutions.holistic   #todo el cuerpo
mp_drawing = mp.solutions.drawing_utils # DIBUJA LOS PUNTOS DEL CUERPO

In [74]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #CONVERSION COLOR BGR 2 RGB
    image.flags.writeable = False                  # Escritura de imagen anulada(False)
    results = model.process(image)                 # Hacer deteccion
    image.flags.writeable = True                   # Escritura de imagen activada(True) 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # CONVERSION COLOR RGB 2 BGR
    return image, results

DIBUJA PUNTOS DE LAS DIFERENTES PARTES

In [75]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, FACEMESH_CONTOURS) 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 

DIBUJA CONEXIONES ENTRE PUNTOS 

In [76]:
def draw_styled_landmarks(image, results):
    # conexiones cara
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # conexiones cuerpo
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # conexiones mano izquierda
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # conexiones mano derecha  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

ACTIVAR CAMARA Y DETECCIONES

In [106]:

cap = cv2.VideoCapture(0)   # inicializar captura puerto 0

# Inicializar modelo holistico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # lectura de camara
        ret, frame = cap.read()

        # Hacer detecciones
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Dibujar landmarks( puntos y rayas)
        draw_styled_landmarks(image, results)

        # Ver en pantalla
        cv2.imshow('OpenCV Feed', image)

        # Para apagar imagen,  (0XFF)es codigo para 64bits
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()   # finalizar captura
    cv2.destroyAllWindows()  # cerrar cualquier ventana abierta

In [78]:
draw_landmarks(frame, results) # dibuja los puntos del video)

COMPROBACIONES

In [84]:
#plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) # ver captura de video

In [59]:
#results.pose_landmarks.landmark[0].visibility

0.9999752640724182

In [85]:
#len(results.left_hand_landmarks.landmark) # puntos que ha detectado en la captura)
                                            

EXTRAER VALORES DE LOS PUNTOS CLAVE

In [81]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [82]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [91]:
# face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() 
#     if results.face_landmarks 
#     else np.zeros(1404)


In [86]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [87]:
result_test = extract_keypoints(results)

In [88]:
result_test

array([ 0.56203115,  0.65295368, -0.36940938, ...,  0.        ,
        0.        ,  0.        ])

In [89]:
#grabacion test(directorio jupyter)
np.save('0', result_test)

In [90]:
#cargar test
np.load('0.npy')

array([ 0.56203115,  0.65295368, -0.36940938, ...,  0.        ,
        0.        ,  0.        ])

CONFIGURAR CARPETAS PARA SELECCION DE ELEMENTOS

In [98]:
# DONDE BUSCAR LOS DATOS
DATA_PATH = os.path.join('./reconocimiento_imagenes/DATA/') 

# ACCIONES A DETECTAR
actions = np.array(['colegio', 'sentir', 'hombre'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [103]:
#creamos 30 carpetas de cada elemento en la carpeta asignada, luego anulo la opcion este bloque y continuo

# for action in actions: 
#     for sequence in range(no_sequences):
#         try: 
#             os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
#         except:
#             pass

In [104]:
# #al haber creado las carpetas antes, ya no da error esto
# # esto tambien hay que anularlo porque crea las carpetas mal despues.
# for action in actions: 
#     dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
#     for sequence in range(1,no_sequences+1):
#         try: 
#             os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
#         except:
#             pass

RECOPILACION DE IMAGENES PARA LUEGO TRATARLAS MAS ADELANTE

En cuanto actives la recopilacion, empezara por la primera que has marcado 
en mi caso (colegio) y se iran grabando fotogramas y seguira grabando hasta 
que llege a 30, luego seguira con la siguiente imagen (sentir), a asi sucesivamente hasta que 
se acaben las imagenes que hemos marcado.
Esto se grabara en las carpetas que antes hemos creado

In [107]:
cap = cv2.VideoCapture(0)

# Inicializar modelo holistico 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NUEVO BUCLE
    
    # RECORRER ACCIONES
    for action in actions:
        # RECORRER LAS SECUENCIAS DE VIDEO
        for sequence in range(start_folder, start_folder+no_sequences):
            # RECORRER LA DURACION DE LA SECUENCIA DE VIDEO
            for frame_num in range(sequence_length):

                #lectura de camara
                ret, frame = cap.read()

                # Hacer detecciones
                image, results = mediapipe_detection(frame, holistic)

                # Dibujar landmarks
                draw_styled_landmarks(image, results)
                
                
                # ( Aplicar logica en espera)
                if frame_num == 0: 
                    cv2.putText(image, 'COMENZAR COLECCION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Recopilando frames para {} Video Numero {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Mostrar en pantalla
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Recopilando frames para {} Video Numero {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Mostrar en pantalla
                    cv2.imshow('OpenCV Feed', image)
                
                # Exportar puntos clave
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # apagar camara
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [108]:
cap.release()
cv2.destroyAllWindows()

PREPROCESAR DATOS, CREAR ETIQUETAS Y FUNCIONES

In [109]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [110]:
# para ver etiquetas 
label_map = {label:num for num, label in enumerate(actions)}

In [112]:
label_map

{'colegio': 0, 'sentir': 1, 'hombre': 2}

In [113]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [114]:
np.array(sequences).shape

(90, 30, 1662)

In [115]:
np.array(labels).shape

(90,)

In [116]:
X = np.array(sequences)


In [117]:
X.shape

(90, 30, 1662)

In [118]:
y = to_categorical(labels).astype(int)

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [120]:
y_test.shape

(5, 3)

CONSTRUIR Y ENTRENAR LA RED NEURONAL

In [121]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [122]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [136]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [137]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [138]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 2s 45ms/step - loss: 4.8000 - categorical_accuracy: 0.3059
Epoch 2/2000
3/3 [==============================] - 0s 43ms/step - loss: 2.7187 - categorical_accuracy: 0.3765
Epoch 3/2000
3/3 [==============================] - 0s 37ms/step - loss: 1.2590 - categorical_accuracy: 0.5765
Epoch 4/2000
3/3 [==============================] - 0s 42ms/step - loss: 6.3262 - categorical_accuracy: 0.5765
Epoch 5/2000
3/3 [==============================] - 0s 43ms/step - loss: 1.0909 - categorical_accuracy: 0.6235
Epoch 6/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.8611 - categorical_accuracy: 0.6588
Epoch 7/2000
3/3 [==============================] - 0s 43ms/step - loss: 3.8306 - categorical_accuracy: 0.6118
Epoch 8/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.9414 - categorical_accuracy: 0.6471
Epoch 9/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.9456 - categorical_accuracy: 0.5765
E

3/3 [==============================] - 0s 36ms/step - loss: 0.2726 - categorical_accuracy: 0.8706
Epoch 75/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.2399 - categorical_accuracy: 0.8824
Epoch 76/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.3567 - categorical_accuracy: 0.8353
Epoch 77/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.4095 - categorical_accuracy: 0.8000
Epoch 78/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.6524 - categorical_accuracy: 0.8235
Epoch 79/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.3392 - categorical_accuracy: 0.8706
Epoch 80/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.2969 - categorical_accuracy: 0.8824
Epoch 81/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.3956 - categorical_accuracy: 0.8235
Epoch 82/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.2698 - categorical_accuracy: 0.8471
Epoch 

3/3 [==============================] - 0s 42ms/step - loss: 0.2834 - categorical_accuracy: 0.8588
Epoch 147/2000
3/3 [==============================] - 0s 48ms/step - loss: 17.9359 - categorical_accuracy: 0.7412
Epoch 148/2000
3/3 [==============================] - 0s 34ms/step - loss: 29.6586 - categorical_accuracy: 0.8000
Epoch 149/2000
3/3 [==============================] - 0s 36ms/step - loss: 6.5243 - categorical_accuracy: 0.7176
Epoch 150/2000
3/3 [==============================] - 0s 43ms/step - loss: 38.4505 - categorical_accuracy: 0.4471
Epoch 151/2000
3/3 [==============================] - 0s 38ms/step - loss: 51.5542 - categorical_accuracy: 0.5176
Epoch 152/2000
3/3 [==============================] - 0s 42ms/step - loss: 87.4782 - categorical_accuracy: 0.5647
Epoch 153/2000
3/3 [==============================] - 0s 42ms/step - loss: 181.9700 - categorical_accuracy: 0.3647
Epoch 154/2000
3/3 [==============================] - 0s 43ms/step - loss: 71.4514 - categorical_accurac

3/3 [==============================] - 0s 40ms/step - loss: 199.0880 - categorical_accuracy: 0.4471
Epoch 218/2000
3/3 [==============================] - 0s 43ms/step - loss: 301.1883 - categorical_accuracy: 0.3294
Epoch 219/2000
3/3 [==============================] - 0s 35ms/step - loss: 221.9440 - categorical_accuracy: 0.4353
Epoch 220/2000
3/3 [==============================] - 0s 36ms/step - loss: 181.0348 - categorical_accuracy: 0.4000
Epoch 221/2000
3/3 [==============================] - 0s 37ms/step - loss: 219.5349 - categorical_accuracy: 0.3529
Epoch 222/2000
3/3 [==============================] - 0s 38ms/step - loss: 154.5215 - categorical_accuracy: 0.3176
Epoch 223/2000
3/3 [==============================] - 0s 40ms/step - loss: 91.6840 - categorical_accuracy: 0.2824
Epoch 224/2000
3/3 [==============================] - 0s 41ms/step - loss: 68.2103 - categorical_accuracy: 0.3647
Epoch 225/2000
3/3 [==============================] - 0s 36ms/step - loss: 49.9698 - categorical_

3/3 [==============================] - 0s 36ms/step - loss: 0.9263 - categorical_accuracy: 0.4706
Epoch 290/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.9248 - categorical_accuracy: 0.4706
Epoch 291/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.9232 - categorical_accuracy: 0.4706
Epoch 292/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.9217 - categorical_accuracy: 0.4706
Epoch 293/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.9202 - categorical_accuracy: 0.4824
Epoch 294/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.9186 - categorical_accuracy: 0.4706
Epoch 295/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.9172 - categorical_accuracy: 0.4118
Epoch 296/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.9158 - categorical_accuracy: 0.4235
Epoch 297/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.9143 - categorical_accuracy: 0.517

...

3/3 [==============================] - 0s 35ms/step - loss: 0.5564 - categorical_accuracy: 0.6941
Epoch 1658/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.5558 - categorical_accuracy: 0.6941
Epoch 1659/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.5554 - categorical_accuracy: 0.6941
Epoch 1660/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.5551 - categorical_accuracy: 0.6941
Epoch 1661/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.5545 - categorical_accuracy: 0.6941
Epoch 1662/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.5544 - categorical_accuracy: 0.6941
Epoch 1663/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.5543 - categorical_accuracy: 0.6941
Epoch 1664/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.5539 - categorical_accuracy: 0.6941
Epoch 1665/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.5536 - categorical_accurac

3/3 [==============================] - 0s 43ms/step - loss: 0.5403 - categorical_accuracy: 0.7176
Epoch 1730/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.5404 - categorical_accuracy: 0.7176
Epoch 1731/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.5403 - categorical_accuracy: 0.7176
Epoch 1732/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.5403 - categorical_accuracy: 0.7176
Epoch 1733/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.5397 - categorical_accuracy: 0.7059
Epoch 1734/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.5398 - categorical_accuracy: 0.7059
Epoch 1735/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.5396 - categorical_accuracy: 0.7059
Epoch 1736/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.5394 - categorical_accuracy: 0.7059
Epoch 1737/2000
3/3 [==============================] - 0s 48ms/step - loss: 0.5394 - categorical_accurac

3/3 [==============================] - 0s 45ms/step - loss: 0.5251 - categorical_accuracy: 0.7176
Epoch 1802/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.5213 - categorical_accuracy: 0.7176
Epoch 1803/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.5229 - categorical_accuracy: 0.7529
Epoch 1804/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.5173 - categorical_accuracy: 0.7765
Epoch 1805/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.5194 - categorical_accuracy: 0.7294
Epoch 1806/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.5197 - categorical_accuracy: 0.7294
Epoch 1807/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.5152 - categorical_accuracy: 0.7412
Epoch 1808/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.5129 - categorical_accuracy: 0.7412
Epoch 1809/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.5223 - categorical_accurac

3/3 [==============================] - 0s 44ms/step - loss: 0.5446 - categorical_accuracy: 0.7294
Epoch 1874/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.5789 - categorical_accuracy: 0.6824
Epoch 1875/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.5945 - categorical_accuracy: 0.6588
Epoch 1876/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.5691 - categorical_accuracy: 0.7059
Epoch 1877/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.5418 - categorical_accuracy: 0.7059
Epoch 1878/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.5397 - categorical_accuracy: 0.6471
Epoch 1879/2000
3/3 [==============================] - 0s 53ms/step - loss: 0.5322 - categorical_accuracy: 0.6588
Epoch 1880/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.5199 - categorical_accuracy: 0.7529
Epoch 1881/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.5108 - categorical_accurac

3/3 [==============================] - 0s 39ms/step - loss: 0.6208 - categorical_accuracy: 0.6471
Epoch 1946/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.5941 - categorical_accuracy: 0.6824
Epoch 1947/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.5729 - categorical_accuracy: 0.6353
Epoch 1948/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.5588 - categorical_accuracy: 0.6353
Epoch 1949/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.5697 - categorical_accuracy: 0.6235
Epoch 1950/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.5681 - categorical_accuracy: 0.6235
Epoch 1951/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.5435 - categorical_accuracy: 0.6353
Epoch 1952/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.5350 - categorical_accuracy: 0.6588
Epoch 1953/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.5347 - categorical_accurac

In [145]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

PREDICCIONES

In [127]:
res = model.predict(X_test)

In [128]:
actions[np.argmax(res[4])]

'sentir'

In [129]:
actions[np.argmax(y_test[4])]

'hombre'

9. Save Weights

In [146]:
#GRABAR MODELO CON EL NOMBRE ACTION.H5
model.save('action.h5')

In [131]:
#CUIDADO, BORRA EL ANTERIOR MODELO 
del model

In [139]:
#CARGAR EL MODELO NUEVO
model.load_weights('action.h5')

EVALUACION ( Confusion Matrix y Accuracy)

In [157]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [158]:
yhat = model.predict(X_test)

In [159]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [143]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [1, 1]],

       [[1, 2],
        [0, 2]],

       [[4, 0],
        [1, 0]]], dtype=int64)

In [144]:
accuracy_score(ytrue, yhat)

0.6

TESTEAR EN TIEMPO REAL

In [147]:
from scipy import stats

In [150]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


In [152]:
# plt.figure(figsize=(18,18))
# plt.imshow(prob_viz(res, actions, image, colors))

In [164]:
# 1. NUEVA DETECCION DE VARIABLES
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# inicializar holistic
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Lectura de camara
        ret, frame = cap.read()

        # Hacer detecciones
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Dibujar landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # mostrar en pantalla
        cv2.imshow('OpenCV Feed', image)

        # parar camara
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
